In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/content/drive/MyDrive/fashion_product_images"
%cd "/content/drive/MyDrive/fashion_product_images"

/content/drive/.shortcut-targets-by-id/1-0ODWkYUqKds7i6A9u5GtxUYKvjnHaS_/fashion_product_images


In [ ]:
!kaggle datasets download -d paramaggarwal/fashion-product-images-small

Dataset URL: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small
License(s): copyright-authors
fashion-product-images-small.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip \*.zip && rm *.zip

Streaming output truncated to the last 5000 lines.
  inflating: myntradataset/images/5813.jpg  
  inflating: myntradataset/images/58131.jpg  
  inflating: myntradataset/images/58132.jpg  
  inflating: myntradataset/images/58133.jpg  
  inflating: myntradataset/images/58135.jpg  
  inflating: myntradataset/images/58136.jpg  
  inflating: myntradataset/images/58137.jpg  
  inflating: myntradataset/images/58138.jpg  
  inflating: myntradataset/images/58139.jpg  
  inflating: myntradataset/images/5814.jpg  
  inflating: myntradataset/images/58140.jpg  
  inflating: myntradataset/images/58141.jpg  
  inflating: myntradataset/images/58143.jpg  
  inflating: myntradataset/images/58144.jpg  
  inflating: myntradataset/images/58145.jpg  
  inflating: myntradataset/images/58146.jpg  
  inflating: myntradataset/images/58147.jpg  
  inflating: myntradataset/images/58148.jpg  
  inflating: myntradataset/images/58149.jpg  
  inflating: myntradataset/images/5815.jpg  
  inflating: myntradataset/image

In [ ]:
# Import necessary libraries
from flask import Flask, request, jsonify
from flask_cors import CORS
import base64
from sklearn.neighbors import NearestNeighbors

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Load the model and precomputed features
with open('embeddings.pkl', 'rb') as f:
    feature_list = pickle.load(f)

with open('filenames.pkl', 'rb') as f:
    filenames = pickle.load(f)

feature_list = np.array(feature_list)
neighbors = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')
neighbors.fit(feature_list)

def extract_features(img_path, model):
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    return result

@app.route('/', methods=['GET'])
def home():
    return 'success', 200

@app.route('/find_similar', methods=['POST'])
def find_similar():
    if 'file' not in request.files:
        return 'No file part', 400

    file = request.files['file']
    if file.filename == '':
        return 'No selected file', 400

    if file:
        # Ensure 'uploads' directory exists
        if not os.path.exists('uploads'):
            os.makedirs('uploads')

        filepath = os.path.join('uploads', file.filename)
        file.save(filepath)

        feature = extract_features(filepath, model)
        distances, indices = neighbors.kneighbors([feature])

        similar_files = [filenames[i] for i in indices[0][1:6]]
        print(similar_files)
        # Convert images to base64
        similar_images_base64 = []
        for filename in similar_files:
            with open(filename, "rb") as image_file:
                encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
                similar_images_base64.append(encoded_string)

        return jsonify(similar_images_base64)

# Run Flask app with ngrok
app.run(port=80)

In [ ]:
!pip install flask flask-cors pyngrok

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalMaxPooling2D, Dense, Dropout, Input
from tensorflow.keras.models import Model
import pickle
from tqdm import tqdm

# Define the input shape
input_shape = (224, 224, 3)

# Load the base model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze all layers except the last 10 layers
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Build the model using the Functional API
inputs = Input(shape=input_shape)
x = base_model(inputs, training=False)  # Pass inputs through the base model
x = GlobalMaxPooling2D()(x)  # Apply pooling
x = Dense(512, activation='relu')(x)  # Add a Dense layer
x = Dropout(0.5)(x)  # Add Dropout

# Define the output layer (if you need a specific output layer, adjust here)
outputs = Dense(10, activation='softmax')(x)  # Example for classification

# Create the model
model = Model(inputs, outputs)

# Summary of the model
model.summary()

# Image directory and batch size
image_directory = 'images'  # Replace with your image directory
batch_size = 32

# Custom image generator without augmentation for feature extraction
def custom_image_generator(directory, batch_size=32, target_size=(224, 224)):
    files = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    num_files = len(files)

    while True:
        for offset in range(0, num_files, batch_size):
            batch_files = files[offset:offset + batch_size]
            batch_images = []

            for file in batch_files:
                img = load_img(file, target_size=target_size)
                img_array = img_to_array(img)
                img_array = preprocess_input(img_array)
                batch_images.append(img_array)

            batch_images = np.array(batch_images)
            yield batch_images, batch_files

# Generator initialization
generator = custom_image_generator(image_directory, batch_size=batch_size)

# Calculate number of batches
num_images = len([name for name in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, name))])
num_batches = np.ceil(num_images / batch_size)

# Initialize lists to store features and filenames
feature_list = []
filenames = []

# Extract features in batches
for _ in tqdm(range(int(num_batches))):
    batch_images, batch_files = next(generator)
    batch_features = model.predict(batch_images)
    batch_features = [feature.flatten() for feature in batch_features]  # Flatten features
    feature_list.extend(batch_features)
    filenames.extend(batch_files)

# Save features and filenames
with open('embeddings.pkl', 'wb') as f:
    pickle.dump(feature_list, f)

with open('filenames.pkl', 'wb') as f:
    pickle.dump(filenames, f)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d_1 (Gl  (None, 2048)              0         
 obalMaxPooling2D)                                               
                                                                 
 dense_2 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                           

  0%|          | 0/541 [00:00<?, ?it/s]

1/1 [==============================] - 2s 2s/step


  0%|          | 1/541 [00:21<3:12:54, 21.43s/it]

1/1 [==============================] - 1s 556ms/step


...

1/1 [==============================] - 1s 553ms/step


100%|█████████▉| 540/541 [15:00<00:00,  1.38it/s]

1/1 [==============================] - 1s 1s/step


100%|██████████| 541/541 [15:01<00:00,  1.67s/it]


OSError: [Errno 95] Operation not supported: 'embeddings.pkl'

In [ ]:
# Create the output directory if it doesn't exist
output_directory = 'output'  # Choose an appropriate directory
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Save features and filenames in the output directory
with open(os.path.join(output_directory, 'embeddings.pkl'), 'wb') as f:
    pickle.dump(feature_list, f)

with open(os.path.join(output_directory, 'filenames.pkl'), 'wb') as f:
    pickle.dump(filenames, f)

In [ ]:
!pip install flask flask-cors pyngrok

from flask import Flask, request, jsonify
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalMaxPooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.neighbors import NearestNeighbors
import pickle
import os
from sklearn.metrics.pairwise import cosine_similarity

app = Flask(__name__)

# Load precomputed features and filenames
with open('./output/embeddings.pkl', 'rb') as f:
    feature_list = pickle.load(f)
with open('./output/filenames.pkl', 'rb') as f:
    filenames = pickle.load(f)

feature_array = np.array(feature_list)

# Model setup
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers[:-10]:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalMaxPooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5)
])

# Fit the Nearest Neighbors model
nn_model = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')
nn_model.fit(feature_array)

# Define the function to get top 5 recommendations using Nearest Neighbors
def get_top_5_recommendations_nn(image_features, nn_model, filenames):
    distances, indices = nn_model.kneighbors([image_features])
    similar_files = [filenames[i] for i in indices[0][1:6]]
    return similar_files

# Define feature extraction function
def extract_features(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    return features.flatten()

# Define similarity function
def get_top_5_recommendations(image_features):
    # Compute cosine similarity between the input image features and precomputed features
    similarities = cosine_similarity([image_features], feature_array)
    # Get indices of top 5 most similar images
    top_indices = similarities[0].argsort()[-5:][::-1]
    return [filenames[i] for i in top_indices]

# Define API endpoint for recommendation
@app.route('/recommend', methods=['POST'])
def recommend():
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No file selected'}), 400

    # Save the uploaded file
    file_path = 'temp_image.jpg'
    file.save(file_path)

    # Extract features from the uploaded image
    uploaded_image_features = extract_features(file_path)

    # Remove the temporary file
    os.remove(file_path)

    # Check for query parameter to choose the recommendation method
    method = request.args.get('method', 'cosine')

    if method == 'nn':
        # Get top 5 recommendations using Nearest Neighbors
        recommendations = get_top_5_recommendations_nn(uploaded_image_features, nn_model, filenames)
    else:
        # Default to cosine similarity
        recommendations = get_top_5_recommendations(uploaded_image_features)

    return jsonify({'recommendations': recommendations})

if __name__ == '__main__':
    app.run(debug=True)


  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached Flask_Cors-4.0.1-py2.py3-none-any.whl.metadata (5.5 kB)
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
Using cached flask-3.0.3-py3-none-any.whl (101 kB)
Using cached Flask_Cors-4.0.1-py2.py3-none-any.whl (14 kB)
Using cached blinker-1.8.2-py3-none-any.whl (9.5 kB)
  Attempting uninstall: blinker
    Found existing installation: blinker 1.4
error: uninstall-distutils-installed-package

× Cannot uninstall blinker 1.4
╰─> It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


ModuleNotFoundError: No module named 'flask'

In [ ]:
# Read the Ngrok token from the file
with open("/content/drive/MyDrive/ngrok-creds.txt", "r") as token_file:
    ngrok_token = token_file.read().strip()

In [ ]:
!pip install flask-ngrok
!pip install pyngrok
!pip install flask-cors
!pip install flask

from pyngrok import ngrok

# Authenticate Ngrok using the token from the file
ngrok.set_auth_token(ngrok_token)

# Start the tunnel
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")


  Using cached flask_ngrok-0.0.25-py3-none-any.whl.metadata (1.8 kB)
  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
Using cached flask_ngrok-0.0.25-py3-none-any.whl (3.1 kB)
Using cached flask-3.0.3-py3-none-any.whl (101 kB)
Using cached blinker-1.8.2-py3-none-any.whl (9.5 kB)
  Attempting uninstall: blinker
    Found existing installation: blinker 1.4
error: uninstall-distutils-installed-package

× Cannot uninstall blinker 1.4
╰─> It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
  Using cached Flask_Cors-4.0.1-py2.py3-none-any.whl.metadata (5.5 kB)
  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
Using cached Flask_Cors-4.0.1-py2.py3-none-any.whl (14 kB)
Using cached flask-3.0.3-py3-none-any.whl (101 kB)
Using cached blinker-1.8.2-py3-